In [1]:
%reload_ext autoreload
%autoreload 2

https://gist.github.com/endolith/255291

In [2]:
import time
import datetime
import uuid
import pyageng
from scipy import signal
from skimage import util
import copy

In [3]:
import pyarrow.parquet as pq
import os
import seaborn as sns
import numpy.fft as fft

In [4]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

/home/walle/miniconda3/envs/fastai-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  


In [5]:
from fastai import *
from fastai.tabular import *
from fastai.utils import *

In [6]:
import fastai
print(fastai.__version__)

1.0.39.dev0


In [7]:
torch.cuda.set_device(1)

In [8]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [10]:
DATE = datetime.datetime.today().strftime('%Y%m%d')
UID=str(uuid.uuid4())[:8]

In [11]:
DATE

'20181227'

In [12]:
UID

'a4011564'

## Data preparation

In [13]:
path = Path('../input/')
bad_path = path/'train/bad/'
good_path = path/'train/good/'

In [14]:
test_path = path/'test/'

In [15]:
meta_train = pd.read_csv(path/'metadata_train.csv')

In [16]:
meta_train.head(n=2)

,signal_id,id_measurement,phase,target
0,0,0,0,0
1,1,0,1,0


In [17]:
meta_train.shape

(8712, 4)

In [18]:
#meta_train_0_900 = meta_train[:900]

In [19]:

# Start small, load in 900 signals and calculate some basic aggregates (mean, sum)
# Read in 900 signals
# Each column contains one signal
#subset_train_0_900 = pq.read_pandas(path/'train.parquet', columns=[str(i) for i in range(900)]).to_pandas()

In [20]:
meta_train.head(n=2)

,signal_id,id_measurement,phase,target
0,0,0,0,0
1,1,0,1,0


In [21]:
features = meta_train.columns

In [22]:
features

Index(['signal_id', 'id_measurement', 'phase', 'target'], dtype='object')

In [23]:
meta_test = pd.read_csv(path/'metadata_test.csv')

### Parquet Data

In [24]:
gc.collect()
start_time = time.time()
train_df = pq.read_pandas(path/'train.parquet').to_pandas()
end_time = time.time()
print(f'loading took {end_time-start_time} secs')

loading took 4.233912467956543 secs


In [25]:
nan = 0
for col in range(len(train_df.columns)):
    nan += np.count_nonzero(train_df.loc[col, :].isnull())
print("train.parquet has {} nulls".format(nan))
print("train.parquet shape: {}".format(train_df.shape))

train.parquet has 0 nulls
train.parquet shape: (800000, 8712)


In [26]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,8462,8463,8464,8465,8466,8467,8468,8469,8470,8471,8472,8473,8474,8475,8476,8477,8478,8479,8480,8481,8482,8483,8484,8485,8486,8487,8488,8489,8490,8491,8492,8493,8494,8495,8496,8497,8498,8499,8500,8501,8502,8503,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513,8514,8515,8516,8517,8518,8519,8520,8521,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531,8532,8533,8534,8535,8536,8537,8538,8539,8540,8541,8542,8543,8544,8545,8546,8547,8548,8549,8550,8551,8552,8553,8554,8555,8556,8557,8558,8559,8560,8561,8562,8563,8564,8565,8566,8567,8568,8569,8570,8571,8572,8573,8574,8575,8576,8577,8578,8579,8580,8581,8582,8583,8584,8585,8586,8587,8588,8589,8590,8591,8592,8593,8594,8595,8596,8597,8598,8599,8600,8601,8602,8603,8604,8605,8606,8607,8608,8609,8610,8611,8612,8613,8614,8615,8616,8617,8618,8619,8620,8621,8622,8623,8624,8625,8626,8627,8628,8629,8630,8631,8632,8633,8634,8635,8636,8637,8638,8639,8640,8641,8642,8643,8644,8645,8646,8647,8648,8649,8650,8651,8652,8653,8654,8655,8656,8657,8658,8659,8660,8661,8662,8663,8664,8665,8666,8667,8668,8669,8670,8671,8672,8673,8674,8675,8676,8677,8678,8679,8680,8681,8682,8683,8684,8685,8686,8687,8688,8689,8690,8691,8692,8693,8694,8695,8696,8697,8698,8699,8700,8701,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711
0,18,1,-19,-16,-5,19,-15,15,-1,-16,-9,23,-1,-17,16,-8,17,-10,21,-8,-12,-7,23,-15,-20,13,4,-20,1,15,13,-2,-17,5,16,-20,9,-16,7,10,-17,5,-16,18,0,-8,17,-13,-6,18,-17,16,4,-19,-18,-3,21,-19,14,3,-3,-18,18,-4,22,-19,-16,14,-6,3,12,-18,-16,19,-10,13,-21,5,11,15,-24,13,9,-25,-18,-7,23,12,6,-18,15,1,-18,10,7,-18,19,-17,-1,21,-16,-5,-21,6,14,-6,16,-16,17,-13,-16,11,-21,2,16,-7,-10,1,14,-20,-8,21,-12,11,-18,7,-16,16,1,10,11,-20,-11,-11,22,14,-19,5,-6,21,-15,-13,-8,19,-1,-1,-2,-2,-17,22,9,12,-22,-18,14,10,2,14,-19,-14,17,-7,-6,12,-15,-22,10,13,16,2,-20,9,12,-24,14,-22,5,-1,23,-9,13,-21,7,-14,22,-5,-7,-13,21,13,3,-19,-7,-11,19,-9,17,-10,-18,8,11,-22,14,8,-17,1,17,14,0,-17,0,-16,19,-18,-1,14,-4,-20,18,-19,19,-2,20,-7,-18,19,-11,-6,4,10,-23,18,-16,-8,7,-17,11,11,-19,8,9,8,-22,17,0,-17,0,-18,15,-3,...,15,16,4,-22,20,-2,-16,18,-8,-9,17,-14,-7,-23,6,15,11,2,-19,-5,20,-19,-25,15,4,15,-20,5,-1,16,-15,-2,21,-13,14,-15,-7,-4,-12,20,16,-4,-16,-17,-5,21,-2,18,-16,16,-1,-16,17,-21,2,16,-20,3,3,-22,14,7,-19,15,9,-20,16,-21,8,18,-18,13,2,-14,-11,21,-1,-15,22,-6,15,-17,10,15,-20,10,-21,9,-18,-5,22,24,-5,-16,12,-9,-16,-8,17,-12,-19,15,7,-14,19,-7,-10,18,-9,8,11,-19,3,15,-21,20,-5,-18,16,-13,-6,17,-2,-15,-5,-18,20,-21,7,10,10,-2,-23,-12,-15,21,-20,13,-1,21,-15,-3,17,0,-18,-16,-11,14,-14,19,-7,19,-8,-16,12,-26,13,-14,-6,15,11,3,-20,-9,-8,25,17,-8,-10,20,-15,1,-9,-16,17,8,-20,11,12,-22,15,19,-16,-8,-8,19,-12,-20,8,9,12,2,-19,-14,-4,20,-19,13,2,-19,-5,19,0,11,-19,-26,16,6,-20,3,19,17,-16,-5,1,19,-18,1,13,-19,6,12,-18,-8,18,-9,19,-7,-12,-1,-3,-3,-19,8,3,-12,18,-6,-11,-5,18,-22,12,8,13,6,-21,-15,-9,20
1,18,0,-19,-17,-6,19,-17,16,0,-15,-10,22,0,-16,17,-7,17,-10,20,-9,-12,-7,24,-14,-20,14,4,-20,1,15,14,-3,-18,5,16,-20,9,-15,7,9,-18,5,-17,18,0,-10,15,-14,-5,19,-16,16,5,-19,-18,-4,21,-21,14,3,-5,-18,20,-6,20,-21,-10,21,1,4,13,-17,-14,20,-8,9,-21,5,11,15,-23,15

convert columns to rows

In [27]:
train_df = train_df.T

In [28]:
train_df.shape

(8712, 800000)

#### test data

In [29]:
gc.collect()
start_time = time.time()
test_df = pq.read_pandas(path/'test.parquet').to_pandas()
end_time = time.time()
print(f'loading took {end_time-start_time} secs')

loading took 11.320717334747314 secs


In [30]:
test_df = test_df.T

In [31]:
test_df.shape

(20337, 800000)

In [32]:
y=train_df.iloc[0, :]

In [33]:
len(y)

800000

In [34]:
samples = 800000

800000 samples every 20 millisecond

In [35]:
total_time_sec = 20/1000
total_time_sec

0.02

In [36]:
sr = total_time_sec/samples
sr

2.5e-08

super fast sample rate

#### spectrogram

https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch04.html

In [37]:
def y_to_spec_sk(y):
    #For each slice, calculate the DFT, which returns both positive and negative frequencies 
    #(more on that in “Frequencies and Their Ordering”), so we slice out the positive M2 frequencies for now.
    M = 1024

    slices = util.view_as_windows(y, window_shape=(M,), step=100)
    #print(f'y shape: {y.shape}, Sliced y shape: {slices.shape}')
    win = np.hanning(M + 1)[:-1]
    slices = slices * win
    slices = slices.T
    #print('Shape of `slices`:', slices.shape)
    spectrum = np.fft.fft(slices, axis=0)[:M // 2 + 1:-1]
    spectrum = np.abs(spectrum)
    f, ax = plt.subplots(figsize=(12, 6))

    S = np.abs(spectrum)
    S = 20 * np.log10(S / np.max(S))

    ax.imshow(S, origin='lower', cmap='viridis',
              extent=(0, len(y), 0, sr / 2 / 1000))
    ax.axis('tight')
    ax.set_ylabel('Frequency [kHz]')
    ax.set_xlabel('Time [s]');
    

In [39]:
def y_to_sprectrogram(y, plot_labels=False, is_test=False, out_pixels=300):
    M = 1024
    dpi=100
    
    freqs, times, Sx = signal.spectrogram(y, fs=sr, window='hanning',
                                          nperseg=1024, noverlap=M - 100,
                                          detrend=False, scaling='spectrum')
    f, ax = plt.subplots(figsize=(out_pixels/dpi, out_pixels/dpi), dpi=dpi)
    #TODO set colour to use max/min limit
    #Sx : ndarray; Spectrogram of x. By default, the last axis of Sx corresponds to the segment times.
    print(f'Sx.shape: {Sx.shape}')
    ax.pcolormesh(times, freqs / 1000, 10 * np.log10(Sx), cmap='viridis')
    if plot_labels:
        ax.set_ylabel('Frequency [kHz]')
        ax.set_xlabel('Time [s]')
    else:
        plt.ioff()
        plt.axis('off')
    ax.set_ylim(top=1.2499999999999999e-11)
    plt.show()

In [ ]:
y=train_df.iloc[0, :]


In [ ]:
y_to_sprectrogram(y.values)

In [ ]:
def psd(y):
    freqs, psd = signal.welch(y)
    plt.figure(figsize=(5, 4))
    plt.semilogx(freqs, psd)
    plt.title('PSD: power spectral density')
    plt.xlabel('Frequency')
    plt.ylabel('Power')
    plt.tight_layout()

In [ ]:
y=train_df.iloc[0, :]
psd(y)